In [48]:
from pyspark.sql.types import *
import pandas as pd
import pyspark 
import os 
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_replace, col, udf, rank, asc, sum as spark_sum
from pyspark.sql import functions as F
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql.window import Window

from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.classification import RandomForestClassifier, LogisticRegression, GBTClassifier
from pyspark.ml import Pipeline
from pyspark.ml.stat import ChiSquareTest
from pyspark.sql import Row

In [49]:
# Start a Spark session
spark = SparkSession.builder.appName("s33ding").getOrCreate()
# Read the Parquet file into a DataFrame
df = spark.read.parquet("dataset/enem.parquet")
df.dtypes

[('NU_ANO', 'int'),
 ('NU_INSCRICAO', 'bigint'),
 ('TP_FAIXA_ETARIA', 'string'),
 ('Idade_Calculada', 'int'),
 ('TP_SEXO', 'string'),
 ('TP_ESTADO_CIVIL', 'string'),
 ('TP_COR_RACA', 'string'),
 ('TP_NACIONALIDADE', 'string'),
 ('TP_ST_CONCLUSAO', 'string'),
 ('TP_ANO_CONCLUIU', 'string'),
 ('TP_ESCOLA', 'string'),
 ('TP_ENSINO', 'string'),
 ('TP_DEPENDENCIA_ADM_ESC', 'string'),
 ('TP_LOCALIZACAO_ESC', 'string'),
 ('TP_SIT_FUNC_ESC', 'string'),
 ('CO_UF_PROVA', 'int'),
 ('SG_UF_PROVA', 'string'),
 ('CO_MUNICIPIO_PROVA', 'string'),
 ('NO_MUNICIPIO_PROVA', 'string'),
 ('NOTA_CN_CIENCIAS_DA_NATUREZA', 'float'),
 ('NOTA_CH_CIENCIAS_HUMANAS', 'float'),
 ('NOTA_LC_LINGUAGENS_E_CODIGOS', 'float'),
 ('NOTA_MT_MATEMATICA', 'float'),
 ('NOTA_REDACAO', 'float'),
 ('NOTA_MEDIA_5_NOTAS', 'double')]

CATEGORICAL

In [46]:


# Read the Parquet file into a DataFrame
df = spark.read.parquet("dataset/enem.parquet")
df.dtypes
results = []

# Define a list with the names of the categorical columns
categorical_columns = ['TP_FAIXA_ETARIA', 'TP_SEXO', 'TP_ESTADO_CIVIL', 'TP_NACIONALIDADE', 
                       'TP_ST_CONCLUSAO', 'TP_ANO_CONCLUIU', 'TP_ESCOLA', 'TP_ENSINO', 
                       'TP_DEPENDENCIA_ADM_ESC', 'TP_LOCALIZACAO_ESC', 'TP_SIT_FUNC_ESC', 
                       'SG_UF_PROVA', 'CO_MUNICIPIO_PROVA']

# Define the target variable
target = 'TP_COR_RACA'

# Assuming 'df' is your DataFrame
for column in categorical_columns:
    # Convert the categorical variable and the target to numeric
    indexer_target = StringIndexer(inputCol=target, outputCol=target+"_index")
    df_indexed_target = indexer_target.fit(df).transform(df)

    indexer_feature = StringIndexer(inputCol=column, outputCol=column+"_index")
    df_indexed = indexer_feature.fit(df_indexed_target).transform(df_indexed_target)

    # Conduct the chi-square test
    assembler = VectorAssembler(
        inputCols=[column+"_index"],
        outputCol="features")

    df_assembled = assembler.transform(df_indexed)

    r = ChiSquareTest.test(df_assembled, "features", target+"_index").head()

    # Store the result in a list and convert numpy types to Python types
    results.append((column, float(r.pValues[0]), float(r.degreesOfFreedom[0]), float(r.statistics[0])))

    # Define the schema
schema = StructType([
    StructField("column", StringType(), True),
    StructField("pValue", FloatType(), True),
    StructField("degreesOfFreedom", DoubleType(), True),
    StructField("statistics", DoubleType(), True)
])

# Convert the results to DataFrame
df_results = spark.createDataFrame(results, schema)

df_results.createOrReplaceTempView("chi_squared_results")
spark.sql("""
    SELECT *,
    RANK() OVER (ORDER BY statistics DESC) as rank
    FROM chi_squared_results
""").show()

df_results.write.mode("overwrite").parquet("data_for_dashboards/featuares_categ_tp_cor_raca/")

Exception in thread "serve-DataFrame" java.net.SocketTimeoutException: Accept timed out
	at java.base/sun.nio.ch.NioSocketImpl.timedAccept(NioSocketImpl.java:708)
	at java.base/sun.nio.ch.NioSocketImpl.accept(NioSocketImpl.java:752)
	at java.base/java.net.ServerSocket.implAccept(ServerSocket.java:675)
	at java.base/java.net.ServerSocket.platformImplAccept(ServerSocket.java:641)
	at java.base/java.net.ServerSocket.implAccept(ServerSocket.java:617)
	at java.base/java.net.ServerSocket.implAccept(ServerSocket.java:574)
	at java.base/java.net.ServerSocket.accept(ServerSocket.java:532)
	at org.apache.spark.security.SocketAuthServer$$anon$1.run(SocketAuthServer.scala:65)
23/06/16 22:46:13 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/06/16 22:46:13 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradatio

+--------------------+------+----------------+------------------+----+
|              column|pValue|degreesOfFreedom|        statistics|rank|
+--------------------+------+----------------+------------------+----+
|  CO_MUNICIPIO_PROVA|   0.0|          8730.0|112434.56683746527|   1|
|         SG_UF_PROVA|   0.0|           130.0| 61781.07546856713|   2|
|TP_DEPENDENCIA_AD...|   0.0|            20.0|12340.469420002684|   3|
|           TP_ESCOLA|   0.0|            10.0|11449.389923838318|   4|
|     TP_FAIXA_ETARIA|   0.0|            40.0| 7474.192419265729|   5|
|  TP_LOCALIZACAO_ESC|   0.0|            10.0| 5037.361454483375|   6|
|     TP_SIT_FUNC_ESC|   0.0|            20.0| 4771.274784582533|   7|
|    TP_NACIONALIDADE|   0.0|            20.0|4064.5254548267576|   8|
|           TP_ENSINO|   0.0|            10.0| 3495.915659554101|   9|
|     TP_ANO_CONCLUIU|   0.0|            75.0| 3441.480956706109|  10|
|     TP_ST_CONCLUSAO|   0.0|            10.0|3189.8778349073636|  11|
|     

In [44]:
# Create SparkSession
spark = SparkSession.builder.getOrCreate()

# Read Parquet file into DataFrame
df = spark.read.parquet("dataset/enem.parquet")

# Select relevant columns
selected_cols = ['NOTA_CN_CIENCIAS_DA_NATUREZA', 'NOTA_CH_CIENCIAS_HUMANAS', 'NOTA_LC_LINGUAGENS_E_CODIGOS', 'NOTA_MT_MATEMATICA', 'NOTA_REDACAO', 'TP_COR_RACA']
selected_df = df.select(selected_cols)

# Handle missing values if necessary

# Convert categorical column to numeric labels
indexer = StringIndexer(inputCol='TP_COR_RACA', outputCol='label')
indexed_df = indexer.fit(selected_df).transform(selected_df)

# Assemble features into a vector
assembler = VectorAssembler(inputCols=['NOTA_CN_CIENCIAS_DA_NATUREZA', 'NOTA_CH_CIENCIAS_HUMANAS', 'NOTA_LC_LINGUAGENS_E_CODIGOS', 'NOTA_MT_MATEMATICA', 'NOTA_REDACAO'], outputCol='features')
assembled_df = assembler.transform(indexed_df)

# Split data into training and testing sets
train_data, test_data = assembled_df.randomSplit([0.7, 0.3], seed=42)

# Train RandomForestClassifier model
rf = RandomForestClassifier(labelCol='label', featuresCol='features')
model = rf.fit(train_data)

# Extract feature importances
importances = model.featureImportances

# Define the feature importances
feature_importances = []
# Print feature importances
feature_importances = list(zip(['NOTA_CN_CIENCIAS_DA_NATUREZA', 'NOTA_CH_CIENCIAS_HUMANAS', 'NOTA_LC_LINGUAGENS_E_CODIGOS', 'NOTA_MT_MATEMATICA', 'NOTA_REDACAO'], importances))
feature_importances.sort(key=lambda x: x[1], reverse=True)
for feature, importance in feature_importances:
    res = (feature, importance)
    print(res)

df = pd.DataFrame(feature_importances, columns=["Features","Importance"])
df.to_parquet('data_for_dashboards/featuares_num_tp_cor_raca/tp_cor_raca.parquet')

('NOTA_MT_MATEMATICA', 0.5857740182655987)
('NOTA_LC_LINGUAGENS_E_CODIGOS', 0.16058507931252256)
('NOTA_CN_CIENCIAS_DA_NATUREZA', 0.1459015499100855)
('NOTA_CH_CIENCIAS_HUMANAS', 0.10459298635425907)
('NOTA_REDACAO', 0.0031463661575339553)
